In [3]:
import sys
import xarray as xr
import numpy as np
import torch
import os
import random 
from tqdm import tqdm 
import pickle 
import warnings
warnings.filterwarnings('ignore')
import random    
random.seed(0)
import scipy.stats as st


ROOT = os.getenv('BONNER_ROOT_PATH')
sys.path.append(ROOT)
from config import CACHE, MAJAJ_DATA   
DATASET = 'majajhong'
SUBJECTS = ['Chabo','Tito']
ALPHA_RANGE = [10**i for i in range(10)]

TRAIN_IDS =  pickle.load(open(os.path.join(ROOT,'model_evaluation/predicting_brain_data/benchmarks','majaj_train_ids'), "rb"))
TEST_IDS =  pickle.load(open(os.path.join(ROOT,'model_evaluation/predicting_brain_data/benchmarks','majaj_test_ids'), "rb"))

In [5]:
len(TEST_IDS)

640

In [ ]:
pickle.load()